In [1]:
import os
import requests
import re
from dotenv import load_dotenv
from tutor import KnowledgeGraph
from langchain.embeddings import OpenAIEmbeddings
from bs4 import BeautifulSoup
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI
from tools import wolfram

In [2]:
load_dotenv()
NEO_PASSWORD = os.getenv("NEO_PASSWORD")
NEO_URI = os.getenv("NEO_URI")
NEO_USERNAME = os.getenv("NEO_USERNAME")

In [3]:
knowledge_graph = KnowledgeGraph(NEO_URI, NEO_USERNAME, NEO_PASSWORD)

c:\Users\Kaden\OneDrive\Professional\Athena\Note Creation\Knowledge Graph\src\app\tutor.py:74: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  self.graph = Neo4jGraph(


In [4]:
llm = ChatOpenAI(
    model="gpt-5",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [5]:
llm_with_tools = llm.bind_tools([wolfram])

In [6]:
llm_with_tools

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002219AFA8D90>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000221AD02C070>, root_client=<openai.OpenAI object at 0x000002219AFA8B50>, root_async_client=<openai.AsyncOpenAI object at 0x00000221AD02C2E0>, model_name='gpt-5', model_kwargs={}, openai_api_key=SecretStr('**********'), max_retries=2), kwargs={'tools': [{'type': 'function', 'function': {'name': 'wolfram', 'description': 'Queries Wolframalpha API to get information on math and science topics', 'parameters': {'properties': {'query': {'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}]}, config={}, config_factories=[])

In [7]:
embeddings = OpenAIEmbeddings()
vector_index = knowledge_graph.create_vector_index(embeddings)
entity_chain = knowledge_graph.create_entity_chain(llm_with_tools)

C:\Users\Kaden\AppData\Local\Temp\ipykernel_17612\2845786025.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [8]:
def conversation():
    question = input("Ask a question about Stat 20 (type 'q' to exit): ")
    print(f"Question: {question}")
    if question.lower() != "q":
        response = knowledge_graph.ask_query(llm_with_tools, vector_index, entity_chain, question)
        print(f"Response: {response}")
        conversation()

In [14]:
wolfram.invoke({'query': 'normal distribution'})

"{'Input': 'normal distribution | mean | μ\\nstandard deviation | σ (positive)', 'Statistical properties': 'mean | μ\\nmode | μ\\nstandard deviation | σ\\nvariance | σ^2\\nskewness | 0', 'Probability density function (PDF)': 'e^(-(x - μ)^2/(2 σ^2))/(sqrt(2 π) σ)', 'Cumulative distribution function (CDF)': 'P (X<=x) = 1/2 erfc((μ - x)/(sqrt(2) σ))', 'Percentiles': '10th | μ - 1.28155 σ\\n25th | μ - 0.67449 σ\\n50th | μ\\n75th | μ + 0.67449 σ\\n90th | μ + 1.28155 σ'}"

In [9]:
conversation()

Question: What is the central limit theorem?


c:\Users\Kaden\OneDrive\Professional\Athena\Note Creation\Knowledge Graph\src\app\tutor.py:199: LangChainDeprecationWarning: The function `remove_lucene_chars` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the function exists in the :meth:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :meth:`~langchain-neo4j` and import as `from :meth:`~langchain_neo4j.vectorstores.neo4j_vector import remove_lucene_chars``.
  words = [el for el in remove_lucene_chars(input).split() if el]
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 4, column: 17, offset: 131} for query: "\n                CALL db.index.fulltext.queryNodes('e

Tool call: {'name': 'wolfram', 'args': {'query': 'central limit theorem'}, 'id': 'call_gXEGWlq9iOmZC7fuoNla373A', 'type': 'tool_call'}
Tool response: content="{'Input interpretation': 'central limit theorem', 'Alternate name': 'limit theorem', 'Basic definition': 'The central limit theorem states that any set of variates with a distribution having a finite mean and variance tends to the normal distribution. This allows statisticians to approximate sets of data with unknown distributions as being normal.', 'Detailed definition': 'Let X_1, X_2, ..., X_N be a set of N independent random variates and each X_i have an arbitrary probability distribution P(x_1, ..., x_N) with mean μ_i and a finite variance σ_i^2. Then the normal form variate\\nX_norm congruent ( sum_(i = 1)^N x_i - sum_(i = 1)^N μ_i)/sqrt( sum_(i = 1)^N σ_i^2)\\nhas a limiting cumulative distribution function which approaches a normal distribution.\\nUnder additional conditions on the distribution of the addend, the probabili